# `CLOUD.jl` - 2D linear advection example

In [1]:
using CLOUD, OrdinaryDiffEq

┌ Info: Precompiling CLOUD [fb992021-99c7-4c2d-a14b-5e48ac4045b2]
└ @ Base loading.jl:1423


Define the physical problem

In [2]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(cos(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

Set discretization parameters

In [3]:
M = 4
p = 5
p_map = 5
form = WeakConservationForm()
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Collocated DGSEM on quadrilaterals

Set up a `DGSEM` scheme with LG quadrature on quadrilaterals of degree `p` with `M` edges in each direction

In [4]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Quad(),
    volume_quadrature_rule=LGQuadrature(),
    facet_quadrature_rule=LGQuadrature(),
    mapping_degree=p_map, N_plot=50)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgsem/", overwrite=true, clear=true)

"results/advection_2d_dgsem/"

Run the solver

In [5]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            487ms / 532.5%          57.3MiB / 2353.9%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual          49.9k    1.60s   61.7%  32.1μs    683MiB   50.6%  14.0KiB
   volume terms         49.9k    707ms   27.3%  14.2μs    328MiB   24.3%  6.73KiB
   facet terms          49.9k    641ms   24.7%  12.9μs    260MiB   19.3%  5.34KiB
   mass matrix solve    49.9k    103ms    4.0%  2.07μs   35.0MiB    2.6%     736B
 extrap solution        49.9k    649ms   25.0%  13.0μs    222MiB   16.4%  4.56KiB
 eval flux              49.9k    192ms    7.4%  3.84μs    109MiB    8.1%  2.24KiB
 eval num flux 

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.0011999302847437096
Conservation (initial/final/diff):


LoadError: DimensionMismatch("second dimension of left factor, 36, does not match first dimension of right factor, 81")

## Modal DG scheme on triangles

Set up a `DGMulti` scheme on triangles of degree `p` with `M` edges in each direction

In [7]:
reference_approximation = ReferenceApproximation(
    DGMulti(p), Tri(), mapping_degree=p_map, N_plot=10)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.1)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_2d_dgmulti/", overwrite=true, clear=true)

"results/advection_2d_dgmulti/"

Run the solver

In [8]:
ode_problem_2 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)

save_solution(ode_problem_2.u0, 0.0, results_path, 0)

CLOUD_reset_timer()
sol = solve(ode_problem_2, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:          178μs / 3285712.2%      10.6KiB / 16794359.2%  

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 eval residual           100k    4.66s   79.7%  46.7μs    824MiB   47.6%  8.46KiB
   mass matrix solve     100k    3.62s   62.0%  36.3μs    396MiB   22.8%  4.06KiB
   volume terms          100k    639ms   10.9%  6.41μs    329MiB   19.0%  3.38KiB
   facet terms           100k    169ms    2.9%  1.70μs   31.4MiB    1.8%     330B
 eval flux               100k    430ms    7.4%  4.32μs    108MiB    6.2%  1.11KiB
 gather ext state        100k    292ms    5.0%  2.92μs    510MiB   29.4%  5.23KiB
 eval num flux 

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [9]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), exact_solution, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.00048784850808591414
Conservation (initial/final/diff):
[-6.5217300401244904e-9][-6.521728651044667e-9][1.3890798233884283e-15]
Energy (initial/final/diff):
[0.12499978051190094][0.12498968050064548][-1.0100011255467778e-5]
